# Passport image to json

In [1]:
# Cài đặt các thư viện cần thiết nếu bạn chưa cài đặt
!pip install passporteye paddleocr opencv-python

In [2]:
# Import các thư viện
import cv2
from paddleocr import PaddleOCR
from passporteye import read_mrz
import json
import matplotlib.pyplot as plt
import os

In [3]:
# 1. Sử dụng PassportEye để trích xuất MRZ (Machine Readable Zone)
def extract_mrz_info(image_path):
    """
    Hàm này sử dụng PassportEye để nhận diện MRZ từ ảnh hộ chiếu và trả về thông tin dạng dictionary.
    """
    mrz = read_mrz(image_path)
    if mrz is not None:
        return mrz.to_dict()
    else:
        return {"error": "MRZ not found"}

In [4]:
# 2. Sử dụng OpenCV để xử lý ảnh
def preprocess_image(image_path):
    """
    Hàm này dùng OpenCV để chuyển ảnh sang ảnh xám và áp dụng thresholding để làm rõ các ký tự.
    """
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Chuyển sang ảnh xám
    _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)  # Áp dụng thresholding
    return thresh_image

In [5]:
# 3. Sử dụng PaddleOCR để nhận diện văn bản từ ảnh
def extract_text_from_image(image):
    """
    Hàm này sử dụng PaddleOCR để nhận diện văn bản từ ảnh.
    """
    ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Sử dụng ngôn ngữ tiếng Anh
    result = ocr.ocr(image, cls=True)  # Nhận diện văn bản
    return result

In [6]:
# 4. Kết hợp mọi thứ lại để trích xuất dữ liệu từ ảnh passport
def extract_info_from_passport(image_path):
    """
    Hàm kết hợp các bước trên để trích xuất thông tin từ ảnh passport.
    Trả về kết quả MRZ và văn bản OCR dưới dạng JSON.
    """
    # Trích xuất MRZ
    mrz_info = extract_mrz_info(image_path)
    
    # Xử lý ảnh với OpenCV
    processed_image = preprocess_image(image_path)
    
    # Trích xuất văn bản với PaddleOCR
    ocr_result = extract_text_from_image(processed_image)
    
    # Tạo kết quả trả về dưới dạng JSON
    structured_data = {
        "mrz_info": mrz_info,
        "ocr_result": ocr_result
    }
    
    return json.dumps(structured_data, ensure_ascii=False, indent=4)

In [19]:
# 5. Chạy thử với ảnh passport
image_path = os.path.join(os.getcwd(), '..', 'store', 'input', 'visa-1.jpg' ) # Đường dẫn đến ảnh passport
# kiểm tra xem ảnh có tồn tại không
if not os.path.exists(image_path):
  print(f"Image not found at {image_path} !!!")
else:
  # Gọi hàm để trích xuất thông tin từ ảnh passport
  result = extract_info_from_passport(image_path)



TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
# 6. In kết quả
print(result)  # In kết quả
# Kết quả sẽ là một JSON chứa thông tin MRZ và văn bản OCR
# Chuyển đổi kết quả JSON thành dictionary để dễ đọc hơn
result_dict = json.loads(result)
# In thông tin MRZ
print("MRZ Information:")
print(json.dumps(result_dict["mrz_info"], ensure_ascii=False, indent=4))
# In thông tin OCR
print("OCR Result:")
for line in result_dict["ocr_result"]:
    for word_info in line:
        print(word_info[1][0])  # In từng từ được nhận diện
# Vẽ ảnh với matplotlib
plt.imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))  # Chuyển đổi màu sắc từ BGR sang RGB
plt.axis('off')  # Tắt trục
plt.show()  # Hiển thị ảnh
# Lưu kết quả vào file JSON
output_path = os.path.join(os.getcwd(), '..', 'store', 'output', 'passport_info.json')  # Đường dẫn lưu file JSON
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=4)  # Lưu kết quả vào file JSON
print(f"Result saved to {output_path}")  # In ra thông báo đã lưu kết quả
# Kết thúc


In [ ]:
# 7. Hiển thị ảnh đã xử lý (tùy chọn)
processed_image = preprocess_image(image_path)
plt.imshow(processed_image, cmap='gray')
plt.title('Processed Image (Grayscale + Thresholding)')
plt.axis('off')  # Tắt hiển thị trục
plt.show()